Ejercicio: curvas ROC buenas y malas

En este ejercicio, crearemos algunas curvas ROC para explicar cómo se verían las curvas ROC buenas y malas.

El objetivo de nuestros modelos es identificar si cada elemento detectado en la montaña es un caminante (verdadero) o un árbol (falso). Echemos un vistazo al conjunto de datos.

In [ ]:
import numpy
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/hiker_or_tree.csv
import graphing # custom graphing code. See our GitHub repo for details
import sklearn.model_selection

# Load our data from disk
df = pandas.read_csv("hiker_or_tree.csv", delimiter="\t")

# Split into train and test
train, test =  sklearn.model_selection.train_test_split(df, test_size=0.5, random_state=1)

# Graph our three features
graphing.histogram(test, label_x="height", label_colour="is_hiker", show=True)
graphing.multiple_histogram(test, label_x="motion", label_group="is_hiker", nbins=12, show=True)
graphing.multiple_histogram(test, label_x="texture", label_group="is_hiker", nbins=12)

Tenemos tres características visuales: altura, movimiento y textura. Nuestro objetivo aquí no es optimizar un modelo, sino explorar las curvas ROC, por lo que trabajaremos solo con una a la vez.

Antes de sumergirse en él, eche un vistazo a las distribuciones anteriores. Podemos ver que deberíamos poder usar la altura para separar a los excursionistas de los árboles con bastante facilidad. El movimiento será un poco más difícil, presumiblemente porque los árboles se mueven con el viento y algunos excursionistas se encuentran sentados. La textura parece muy similar para los excursionistas y los árboles.

Un modelo perfecto
¿Cómo sería un ROC perfecto? Si tuviéramos un modelo perfectamente diseñado, calcularía "0% de probabilidad de excursionista" cuando viera cualquier árbol y "100% de excursionista" cuando viera cualquier excursionista. Esto significa que, mientras el umbral de decisión fuera > 0 % y < 100 %, tendría un rendimiento perfecto. Entre estos umbrales, la tasa de verdaderos positivos siempre sería 1 y la tasa de falsos positivos siempre sería 0.

No se preocupe por los umbrales de exactamente 0 y 1 (100%). En 0 estamos obligando a un modelo a devolver un valor Falso y en 1 lo estamos obligando a devolver Verdadero.

Es casi imposible entrenar un modelo que sea tan perfecto, pero por el bien del aprendizaje, supongamos que lo hemos hecho, prediciendo la etiqueta is_hiker en función de la altura.

In [ ]:
import statsmodels.api

# Create a fake model that is perfect at predicting labels
class PerfectModel:
    def predict(self, x):
        # The perfect model believes that hikers are all
        # under 4m tall
        return 1 / (1 + numpy.exp(80*(x - 4)))
    
model = PerfectModel()

# Plot the model
import graphing
graphing.scatter_2D(test, trendline=model.predict)

Nuestra línea roja es nuestro modelo y nuestros puntos azules son nuestros puntos de datos. En el eje y, 0 significa árbol y 1 significa caminante. Podemos ver que nuestro modelo perfecto pasa por cada punto.

Ahora queremos hacer una curva ROC para este modelo perfecto. Hay formas automatizadas de hacer esto, ¡pero estamos aquí para aprender! No es tan difícil de hacer nosotros mismos. Solo tenemos que dividirlo en pasos.

Recuerde que una curva ROC representa la tasa de verdaderos positivos (TPR) frente a la tasa de falsos positivos (FPR). Hagamos una función que pueda calcularlos por nosotros. Si no sabe lo que significan estos términos, preste atención a los comentarios del código:

In [ ]:
def calculate_tpr_fpr(prediction, actual):
    '''
    Calculates true positive rate and false positive rate

    prediction: the labels predicted by the model
    actual:     the correct labels we hope the model predicts
    '''

    # To calculate the true positive rate and true negative rate we need to know
    # TP - how many true positives (where the model predicts hiker, and it is a hiker)
    # TN - how many true negatives (where the model predicts tree, and it is a tree)
    # FP - how many false positives (where the model predicts hiker, but it was a tree)
    # FN - how many false negatives (where the model predicts tree, but it was a hiker)

    # First, make a note of which predictions were 'true' and which were 'false'
    prediction_true = numpy.equal(prediction, 1)
    prediction_false= numpy.equal(prediction, 0)

    # Now, make a note of which correct results were 'true' and which were 'false'
    actual_true = numpy.equal(actual, 1)
    actual_false = numpy.equal(actual, 0)

    # Calculate TP, TN, FP, and FN
    # The combination of sum and '&' counts the overlap
    # For example, TP calculates how many 'true' predictions 
    # overlapped with 'true' labels (correct answers)
    TP = numpy.sum(prediction_true  & actual_true)
    TN = numpy.sum(prediction_false & actual_false)
    FP = numpy.sum(prediction_true  & actual_false)
    FN = numpy.sum(prediction_false & actual_true)

    # Calculate the true positive rate
    # This is the proportion of 'hiker' labels that are identified as hikers
    tpr = TP / (TP + FN)

    # Calculate the false positive rate 
    # This is the proportion of 'tree' labels that are identified as hikers
    fpr = FP / (FP + TN)

    # Return both rates
    return tpr, fpr

print("Ready!")

Ahora recuerde que para hacer una curva ROC, calcule TPR y FPR para una amplia gama de umbrales. Luego trazamos el TPR en el eje y y el FPR en el eje x.

Primero, hagamos un método corto que pueda calcular el TPR y el FPR para un solo umbral de decisión.

In [ ]:
def assess_model(model_predict, feature_name, threshold):
    '''
    Calculates the true positive rate and false positive rate of the model
    at a particular decision threshold

    model_predict: the model's predict function
    feature_name: the feature the model is expecting
    threshold: the decision threshold to use 
    '''

    # Make model predictions for every sample in the test set
    # What we get back is a probability that the sample is a hiker
    # For example, if we had two samples in the test set, we might
    # get 0.45 and 0.65, meaning the model says there is a 45% chance
    # the first sample is a hiker, and 65% chance the second is a 
    # hiker
    probability_of_hiker = model_predict(test[feature_name])
    
    # See which predictions at this threshold would say hiker
    predicted_is_hiker = probability_of_hiker > threshold

    # calculate the true and false positives rates using our
    # handy method
    return calculate_tpr_fpr(predicted_is_hiker, test.is_hiker)

print("Ready!")

Ahora podemos usarlo en un bucle para crear una curva ROC:

In [ ]:
def create_roc_curve(model_predict, feature="height"):
    '''
    This function creates a ROC curve for a given model by testing it
    on the test set for a range of decision thresholds. An ROC curve has
    the True Positive rate on the x-axis and False Positive rate on the 
    y-axis

    model_predict: The model's predict function
    feature: The feature to provide the model's predict function
    '''

    # Calculate what the true positive and false positive rate would be if
    # we had used different thresholds. 

    #  Make a list of thresholds to try
    thresholds = numpy.linspace(0,1,101)

    false_positive_rates = []
    true_positive_rates = []

    # Loop through all thresholds
    for threshold in thresholds:
        # calculate the true and false positives rates using our
        # handy method
        tpr, fpr = assess_model(model_predict, feature, threshold)

        # save the results
        true_positive_rates.append(tpr)
        false_positive_rates.append(fpr)


    # Graph the result
    # You don't need to understand this code, but essentially we are plotting
    # TPR versus FPR as a line plot
    # -- Prepare a dataframe, required by our graphing code
    df_for_graphing = pandas.DataFrame(dict(fpr=false_positive_rates, tpr=true_positive_rates, threshold=thresholds))
    # -- Generate the plot
    fig = graphing.scatter_2D(df_for_graphing, x_range=[-0.05,1.05])
    fig.update_traces(mode='lines') # Comment our this line if you would like to see points rather than lines
    fig.update_yaxes(range=[-0.05, 1.05])

    # Display the graph
    fig.show()


# Create an roc curve for our model
create_roc_curve(model.predict)

¿Qué estamos viendo aquí?

Excepto en un umbral de 0, el modelo siempre tiene una tasa de verdaderos positivos de 1. También tiene siempre una tasa de falsos positivos de 0, a menos que el umbral se haya establecido en 1. Tenga en cuenta que debido a que hemos dibujado una línea, Parece que hay valores intermedios (como un FPR de 0,5) pero la línea simplemente engaña. Si lo desea, comente fig.update_traces(mode='lines') en la celda anterior y vuelva a ejecutar para ver puntos, en lugar de líneas.

Piénselo: nuestro modelo es perfecto. Usándolo, siempre obtendremos todas las respuestas correctas, colocando todos los puntos en la esquina superior izquierda (a menos que el umbral sea 0 o 1, lo que significa que estamos descartando los resultados del modelo por completo).

peor que el azar
Como contraejemplo para entender la curva ROC, consideremos un modelo que es peor que el azar. De hecho, este modelo obtiene todas las respuestas incorrectas.

Esto no sucede a menudo en el mundo real, por lo que nuevamente tendremos que falsificar este modelo también. Tracemos este modelo falso contra nuestros datos:

In [ ]:
# Create a fake model that gets every single answer incorrect
class VeryBadModel:
    def predict(self, x):
        # This model thinks that all people are over 4m tall 
        # and all trees are shorter
        return 1 / (1 + numpy.exp(-80*(x - 4)))

model = VeryBadModel()

# Plot the model
graphing.scatter_2D(test, trendline=model.predict)

Como puede ver, ¡la línea roja (modelo) va en la dirección equivocada! ¿Cómo se verá esto en una curva ROC?

In [ ]:
# run our code to create the ROC curve
create_roc_curve(model.predict)

Es lo opuesto al modelo perfecto. En lugar de que la línea llegue a la parte superior izquierda del gráfico, llega a la parte inferior derecha. Esto significa que el TPR siempre es 0, no hace nada bien. En este ejemplo particular, también tiene siempre una tasa de falsos positivos de 1, siempre que el umbral sea inferior a 1.

Un modelo no mejor que el azar

Los dos modelos anteriores que hemos visto son muy inusuales. Sin embargo, hemos aprendido que nos gustaría que la curva estuviera lo más cerca posible de la parte superior izquierda del gráfico.

¿Cómo sería un modelo que no hace nada mejor que el azar? Echemos un vistazo tratando de ajustar un modelo a nuestra función de textura. Sabemos que esto no funcionará bien, porque hemos visto que los excursionistas y los árboles tienen el mismo rango de texturas de imagen.

In [ ]:
import statsmodels.api

# This is a helper method that reformats the data to be compatible
# with this particular logistic regression model 
prep_data = lambda x:  numpy.column_stack((numpy.full(x.shape, 1), x))

# Train a logistic regression model to predict hiker based on texture
model = statsmodels.api.Logit(train.is_hiker, prep_data(train.texture)).fit()

# Plot the model
graphing.scatter_2D(test, label_x="texture", label_y="is_hiker", trendline=lambda x: model.predict(prep_data(x)))

Nuestro modelo no es muy bueno: no pasa por un solo punto de datos y probablemente no funcionará mejor que el azar. Esto parece extremo, pero cuando trabajamos con problemas más complicados, a veces puede ser difícil encontrar un patrón real en los datos. ¿Cómo se ve esto en una curva ROC?

In [ ]:
# run our code to create the ROC curve
create_roc_curve(lambda x: model.predict(prep_data(x)), "texture")

¡Es una línea diagonal! ¿Por qué?

Recuerde que el modelo no pudo encontrar una manera de predecir de manera confiable la etiqueta de la característica. Está haciendo una variedad de predicciones, pero son esencialmente conjeturas.

Si tenemos un umbral de 0,5, aproximadamente la mitad de nuestras probabilidades estarán por encima del umbral, lo que significa que aproximadamente la mitad de nuestras predicciones son excursionistas. La mitad de las etiquetas también son senderistas, pero no hay correlación entre las dos. Esto significa que obtendremos aproximadamente la mitad de las etiquetas de excursionistas correctas (TPR = 0.5). También obtendremos aproximadamente la mitad de las etiquetas de excursionistas predichas incorrectas (FPR = 0.5).

Si aumentáramos el umbral a 0,8, predeciría el excursionista el 80 % de las veces. Nuevamente, debido a que esto es aleatorio, identificaría aproximadamente al 80% de los excursionistas correctamente (por casualidad), y también al 80% de los árboles como excursionistas.

Si continuamos con este enfoque para todos los umbrales, lograríamos una línea diagonal.

Un modelo realista
En el mundo real, por lo general logramos modelos que funcionan entre pura casualidad (una línea diagonal) y perfectamente (una línea que toca la esquina superior izquierda).

Construyamos finalmente un modelo más realista. Intentaremos predecir si una muestra es un caminante o no basándonos en el movimiento. Nuestro modelo debería funcionar bien, pero no será perfecto. Esto se debe a que los excursionistas a veces se sientan quietos (como los árboles) y los árboles a veces soplan con el viento (en movimiento, como un excursionista).

In [ ]:
import statsmodels.api

# Train a logistic regression model to predict hiker based on motion
model = statsmodels.api.Logit(train.is_hiker, prep_data(train.motion), add_constant=True).fit()

# Plot the model
graphing.scatter_2D(test, label_x="motion", label_y="is_hiker", trendline=lambda x: model.predict(prep_data(x)))

El modelo (línea roja) parece sensato, aunque sabemos que a veces obtendrá respuestas incorrectas.

Ahora veamos la curva ROC:

In [ ]:
create_roc_curve(lambda x: model.predict(prep_data(x)), "motion")

Podemos ver que la curva se abulta hacia la esquina superior izquierda, lo que significa que funciona mucho mejor que el azar.

Esta es una curva ROC bastante típica para un problema de aprendizaje automático 'fácil' como este. Los problemas más difíciles a menudo ven la línea mucho más similar a una línea diagonal.

Por el contrario, si alguna vez nos encontramos con una línea que sobresalga en la dirección opuesta, hacia abajo a la derecha, sabríamos que lo estamos haciendo peor que el azar, y que algo anda muy mal.

Resumen
¡Lo superamos! Las curvas ROC pueden parecer difíciles al principio, especialmente debido a la terminología con respecto a los positivos verdaderos y falsos. Sin embargo, construimos uno desde cero, aquí para tener una idea de cómo funcionan por dentro. Si le resultó difícil, lea de nuevo lentamente y experimente con algunas de las funciones que creamos. No se preocupe, normalmente podemos usar las bibliotecas existentes para hacer la mayor parte de este trabajo por nosotros.

La moda que es importante recordar con estas curvas es que nos gustaría que nuestra línea estuviera lo más cerca posible de la parte superior izquierda del gráfico. Un modelo que puede hacer esto es identificar correctamente el objetivo (como los excursionistas) la mayor parte del tiempo, sin identificar falsamente el objetivo (etiquetar árboles como excursionistas) muy a menudo.